In [146]:
import pandas as pd
import os
from PIL import Image
from fpdf import FPDF


def add_black_border(img, border_thickness):
    """
    Adds a black border around the image.
    
    Parameters:
    - img: Input image
    - border_thickness: The thickness of the border
    
    Returns:
    - Image with black border
    """
    
    # Calculate the new size of the image with the border
    new_width = img.width + 2 * border_thickness
    new_height = img.height + 2 * border_thickness

    # Create a new black image
    new_img = Image.new("RGB", (new_width, new_height), "black")

    # Paste the original image onto the black background, offset by the border thickness
    new_img.paste(img, (border_thickness, border_thickness))

    return new_img


 




def inches_to_mm(inches_value):
    if isinstance(inches_value, tuple):
        return tuple(inch * 25.4 for inch in inches_value)
    else:
        return inches_value * 25.4

def create_pdf_from_images(titles_csv, image_folder, output_pdf, pdf_size_in_inches=(8.27,11.69), margins_in_inches=(0.39,0.79,0.39,0.79), image_scale=1):
    # Convert PDF size and margins to mm
    pdf_size = inches_to_mm(pdf_size_in_inches)
    margins = inches_to_mm(margins_in_inches)

    # Read titles from the CSV
    titles = pd.read_csv(titles_csv).columns.tolist()
    print(len(titles))

    # Create a PDF
    pdf = FPDF(unit="mm", format=pdf_size)
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_margins(margins[0], margins[1], margins[2])

    for title in titles:
        
        pdf.add_page()
        
        # Convert title spaces to underscores to match image filenames
        title_for_image = title.replace(' ', '_').replace("\'", "")
        
        # Find an image with the modified title in its filename
        image_files = [f for f in os.listdir(image_folder) if title_for_image in f and f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if not image_files:
            print(f"Image not found for title: {title}")
            continue
        
        # Open the image with PIL and apply scaling
        image_path = os.path.join(image_folder, image_files[0])
        img = Image.open(image_path)
        img = img.convert('L')
        img_width, img_height = img.size
        img = img.resize((int(img_width * image_scale), int(img_height * image_scale)), Image.ANTIALIAS)
        img = add_black_border(img, 5)

        # Create a blank PDF page of the specified size
        #pdf.add_page(size=pdf_size)

        # Place the title on the top center of the page
        pdf.set_font("Helvetica", size=12)
        pdf.ln(0)  # Adjust this if you want more space between top margin and title
        pdf.cell(0, 10, txt=title, ln=True, align='C')

        # Save the modified image to a temporary location
        tmp_img_path = f"tmp_img_{title_for_image}.png"
        img.save(tmp_img_path, "PNG")

        # Calculate x position to center the image
        x_centered = (pdf_size[0] - img_width * image_scale / 10) / 2

        # Place the image centered below the title
        pdf.image(tmp_img_path, x=x_centered, y=pdf.get_y(), w=img_width * image_scale / 10, h=img_height * image_scale / 10)  # w and h are in mm. Convert pixels to mm.

        # Place the page number at the bottom of the page
        
        # Place the page number at the bottom of the page
        pdf.set_font("Arial", size=10)
        page_number = str(pdf.page_no())
        page_number_width = pdf.get_string_width(page_number) + 6  # approximate width of the cell including the number. The '+6' can be adjusted if necessary.

        x_centered = (pdf_size[0] - page_number_width) / 2  # center the page number's cell
        y_bottom = pdf_size[1] - margins[3] - 20
        pdf.set_xy(x_centered, y_bottom)
        pdf.cell(page_number_width, 10, txt=page_number, ln=True, align='C')

        # Clean up temporary image
        os.remove(tmp_img_path)

    # Save the final PDF
    pdf.output(output_pdf)



In [147]:
# Use the function
create_pdf_from_images(
    titles_csv="Poems/Vol1/poems_vol1.csv",
    image_folder="Images/Vol1",
    output_pdf="Books/poems_vol1_book.pdf",
    pdf_size_in_inches=(5, 8),
    margins_in_inches=(0.375,.375,0.375,.375),  # Left, Top, Right, Bottom in inches
    image_scale=0.4  # Scale down image to 80% of its original size
)

100


/var/folders/rc/tbvl0f854x1gt02h7y3msl4c0000gn/T/ipykernel_90441/961601539.py:75: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((int(img_width * image_scale), int(img_height * image_scale)), Image.ANTIALIAS)
